# 多智能体计划与写作

## 计划和执行

计划和执行代理通过首先计划要做什么，然后执行子任务来实现目标。 这个想法很大程度上受到[BabyAGI](https://github.com/yoheinakajima/babyagi)和[“Plan-and-Solve”论文](https://arxiv.org/abs/2305.04091)的启发。

规划几乎总是由LLM完成。

执行通常由单独的代理（配备工具）完成。

In [1]:
from langchain_community.utilities import SerpAPIWrapper
import os

serpapi_api_key = os.getenv("SERPAPI_API_KEY")

search = SerpAPIWrapper(serpapi_api_key=serpapi_api_key)
search.run("刘亦菲")

[{'title': '刘亦菲：当年你不想挨着我，今年你不得不挨着我',
  'link': 'https://i.ifeng.com/c/8fXhPPSfjFo',
  'source': '凤凰网',
  'date': '7 hours ago',
  'thumbnail': 'https://serpapi.com/searches/6769777435bad4c37e3c5662/images/e8c3f65d7d71d1ad12ab7f7ce4e83131ee32447dcd12ff2d.jpeg'},
 {'title': '刘亦菲昔日爱豆时期画面被挖 唱跳俱佳不输女团',
  'link': 'https://www.orientaldaily.com.my/news/entertainment/2024/12/23/701292',
  'source': '马来西亚东方日报',
  'date': '13 hours ago',
  'thumbnail': 'https://serpapi.com/searches/6769777435bad4c37e3c5662/images/e8c3f65d7d71d1ad7fd287e2bc73c444e15df3d511db0a0f.jpeg'},
 {'title': '刘亦菲携母亲走机场 妈妈穿碎花裙比女儿惊艳',
  'link': 'https://www.bcbay.com/ent/2024/12/22/941899.html',
  'source': '温哥华港湾',
  'date': '1 day ago',
  'thumbnail': 'https://serpapi.com/searches/6769777435bad4c37e3c5662/images/e8c3f65d7d71d1adb6c933ab8af301f4694a5c3cc32f6ee3.jpeg'},
 {'title': '一张内娱大合照，把刘亦菲杨幂的“江湖地位”...',
  'link': 'https://www.backchina.com/news/2024/12/21/949480.html',
  'source': 'BackChina',
  'date': '2 days ago',
  '

## 加载LLM

首先，让我们加载将用于控制代理的语言模型。

In [2]:
from langchain_openai import ChatOpenAI
import os

api_key = os.getenv("LOCAL_API_KEY")
base_url = os.getenv("LOCAL_API_BASE")

llm = ChatOpenAI(api_key=api_key, base_url=base_url, temperature=0.3, max_tokens=4096)

## 定义工具

接下来，让我们定义一些要使用的工具。让我们编写一个非常简单的 Python 函数来计算传入的单词的长度。

请注意，这里我们使用的函数文档字符串非常重要。

In [3]:
from langchain.agents import tool

@tool
def word_length(word: str) -> int:
    """计算传入的单词的长度"""
    return len(word)

word_length.invoke("hello")

5

In [4]:
import requests

@tool
def get_weather(location: str) -> dict:
    """根据城市获取天气数据"""
    api_key = "SKcA5FGgmLvN7faJi"
    city_codes = {
        "西安": "xian",
        "北京": "beijing",
        "上海": "shanghai",
        "广州": "guangzhou",
        "深圳": "shenzhen",
        "杭州": "hangzhou",
        "成都": "chengdu",
        "武汉": "wuhan"
    }
    
    # 处理城市名称
    location = location.replace("市", "").replace("区", "")
    location = city_codes.get(location, location)  # 如果有映射就用映射，没有就用原名
    
    
    url = f"https://api.seniverse.com/v3/weather/now.json?key={api_key}&location={location}&language=zh-Hans&unit=c"
    try:
        response = requests.get(url)
        #print(location)
        if response.status_code == 200:
            data = response.json()
            #print(data)
            weather = {
                'description':data['results'][0]["now"]["text"],
                'temperature':data['results'][0]["now"]["temperature"]
            }
            return weather
        else:
            raise Exception(f"失败接收天气信息：{response.status_code}")
    except Exception as e:
        return {"description": "获取天气信息失败", "detail": str(e)}
            
get_weather.invoke("西安")

{'description': '阴', 'temperature': '4'}

In [5]:
@tool
def search(keywords: str) -> str:
    """根据关键词使用DuckDuckGo搜索信息并返回"""
    return search.run(keywords)


In [6]:
tools = [word_length, get_weather, search]

## 创建提示

现在让我们创建提示。由于 OpenAI 函数调用针对工具使用进行了微调，因此我们几乎不需要任何关于如何推理或如何输出格式的说明。我们将只有两个输入变量： input 和 agent_scratchpad 。 input 应为包含用户目标的字符串。 agent_scratchpad 应该是包含先前代理工具调用和相应工具输出的消息序列。

In [7]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

promptTemplate = """尽可能的帮助用户回答任何问题。
您可以使用以下工具来帮忙解决问题，如果已经知道答案，也可以直接回答：
{tools}

回复格式说明：
----------------------------------

回复我时，请以以下两种格式之一输出问题：

选项1：如果您希望人类使用工具，请使用此选项。
采用以下JSON模式格式化的回复内容：

```json
{{
    "reason": string, \\ 叙述使用工具的原因
    "action": string, \\ 要使用的工具。 必须是 {tool_names} 之一
    "action_input": string \\ 工具的输入
}}
```

选项2：如果您认为你自己已经有了答案或者已经通过使用工具找到了答案，想直接对人类做出反应，请使用此选项。
采用以下JSON模式格式化的回复内容：

```json
{{
    "action": "Final Answer",
    "answer": string \\ 最终回复问题的答案放在这里！
}}
```

用户输入
----------------------------------
这是用户的输入（请记住通过单个选项，以JSON模式格式化的回复内容，不要回复其他内容）：

{input}
"""
promptTemplate

'尽可能的帮助用户回答任何问题。\n您可以使用以下工具来帮忙解决问题，如果已经知道答案，也可以直接回答：\n{tools}\n\n回复格式说明：\n----------------------------------\n\n回复我时，请以以下两种格式之一输出问题：\n\n选项1：如果您希望人类使用工具，请使用此选项。\n采用以下JSON模式格式化的回复内容：\n\n```json\n{{\n    "reason": string, \\ 叙述使用工具的原因\n    "action": string, \\ 要使用的工具。 必须是 {tool_names} 之一\n    "action_input": string \\ 工具的输入\n}}\n```\n\n选项2：如果您认为你自己已经有了答案或者已经通过使用工具找到了答案，想直接对人类做出反应，请使用此选项。\n采用以下JSON模式格式化的回复内容：\n\n```json\n{{\n    "action": "Final Answer",\n    "answer": string \\ 最终回复问题的答案放在这里！\n}}\n```\n\n用户输入\n----------------------------------\n这是用户的输入（请记住通过单个选项，以JSON模式格式化的回复内容，不要回复其他内容）：\n\n{input}\n'

In [8]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是非常强大的助手，你可以使用各种工具来完成人类交给的问题和任务。"),
    ("user", promptTemplate),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={'agent_scratchpad': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMess

In [9]:
from langchain.tools.render import render_text_description

prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools])
)
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'agent_scratchpad': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Ann

In [10]:
render_text_description(list(tools))

'word_length(word: str) -> int - 计算传入的单词的长度\nget_weather(location: str) -> dict - 根据城市获取天气数据\nsearch(keywords: str) -> str - 根据关键词使用DuckDuckGo搜索信息并返回'

## 将工具绑定到LLM

在这种情况下，我们依赖于 OpenAI 工具调用 LLMs，它将工具作为一个单独的参数，并经过专门训练，知道何时调用这些工具。

要将我们的工具传递给代理，我们只需要将它们格式化为 OpenAI 工具格式并将它们传递给我们的模型。（通过 bind 对函数进行 -ing，我们确保每次调用模型时都传入它们。

## 创建代理

将这些部分放在一起，我们现在可以创建代理。我们将导入最后两个实用程序函数：一个用于格式化中间步骤（代理操作、工具输出对）以输入可发送到模型的消息的组件，以及一个用于将输出消息转换为代理操作/代理完成的组件。

In [11]:
from langchain.agents.json_chat.prompt import TEMPLATE_TOOL_RESPONSE

TEMPLATE_TOOL_RESPONSE = """工具响应：
----------------------------------

{observation}

用户的输入：
----------------------------------
请根据工具的响应判断，是否能够回答问题：

{input}

请根据工具响应的内容，思考接下来的回复。回复格式严格按照前面所说的2种JSON回复格式，
选择其中1种进行回复。请记住通过单个选项，以JSON模式格式化回复内容，不要回复其他内容。
"""

In [12]:
from langchain_core.messages import HumanMessage, BaseMessage, AIMessage
from typing import List


def format_log_to_messages(query, intermediate_steps, template_tool_response):
    """构建思考过程"""
    thoughts = []
    for action, observation in intermediate_steps:
        # 直接使用字符串而不是 action.log
        thoughts.append(AIMessage(content=str(action)))
        human_message = HumanMessage(
            content=template_tool_response.format(
                input=query,
                observation=observation
            )
        )
        thoughts.append(human_message)
    return thoughts

In [13]:
from langchain.agents.agent import AgentOutputParser
from langchain_core.output_parsers.json import parse_json_markdown
from langchain_core.exceptions import OutputParserException
from langchain_core.agents import AgentAction, AgentFinish

import logging

logger = logging.getLogger(__name__)

class JSONAgentOutputParser(AgentOutputParser):
    """以 JSON 格式解析工具调用和最终答案。

     期望输出采用两种格式之一。

     如果输出信号表明应采取行动，
     应采用以下格式。 这将导致 AgentAction
     被退回。

    ```
    {
      "action": "search",
      "action_input": "2+2"
    }
    ```
    如果输出信号表明应给出最终答案，
    应采用以下格式。 这将导致 AgentFinish
    被退回。

    ```
    {
      "action": "Final Answer",
      "answer": "4"
    }
    ```
    """

    
    def parse(self, text):
        try:
            response = parse_json_markdown(text)
            if isinstance(response, list):
                logger.warning(f"Got multiple action responses: {response}")
                response = response[0]
            if response.get("action") == "Final Answer":
                return AgentFinish(
                    return_values={"output": response.get("answer")},
                    log=text
                )
            else:
                return AgentAction(
                    tool=response['action'],
                    tool_input=response.get("action_input", ""),
                    log=text
                )
        except OutputParserException as e:
            raise OutputParserException(f"Failed to parse tool call or final answer: {text}") from e
        
    @property
    def _type(self) -> str:
        return "json-agent"

In [14]:
from langchain_core.runnables import RunnablePassthrough

agent = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_log_to_messages(
            x["input"], 
            x["intermediate_steps"], 
            template_tool_response = TEMPLATE_TOOL_RESPONSE
        )
    )
    | prompt
    | llm
    | JSONAgentOutputParser()
)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_messages(x['input'], x['intermediate_steps'], template_tool_response=TEMPLATE_TOOL_RESPONSE))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'agent_scratchpad': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langc

In [15]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [16]:
agent_executor.invoke({"input": "英国现任首相是谁？"})



> Entering new AgentExecutor chain...
```json
{
    "reason": "搜索当前的英国首相",
    "action": "search",
    "action_input": "英国现任首相"
}
```

RecursionError: maximum recursion depth exceeded

## 任务计划与监督者

In [17]:
planPrompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """让我们首先了解问题并制定解决问题的计划。 
请输出以标题“计划：”开头的计划，然后是编号的步骤列表。 
请制定准确完成任务所需的最少步骤数的计划，计划里不要写上任何答案，也不要将答案放到括号里。

如果任务是一个问题，那么最后一步总是“鉴于采取了上述步骤，请回答用户最初的问题”。
""",
        ),
        ("user", "{input}"),
        #MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [18]:
import re
def planParse(message):
    text = message.content
    steps = [v for v in re.split("\n\s*\d+\. ", text)[1:]]
    return steps

In [19]:
planChain = planPrompt | llm | planParse

In [20]:
planChain.invoke({"input":"写一篇关于历代美国总统的年龄跟发布政策偏好关系"})

['研究历任总统及其发布的政策。',
 '收集有关每位总统的年龄和政策偏好的数据。',
 '分析这些数据，寻找任何模式或趋势。',
 '将结果总结为一个报告。 ',
 '鉴于采取了上述步骤，请回答用户最初的问题。']

In [21]:
splitTaskPrompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """让我们首先了解用户问题或者是计划，并根据所提供的工具判断是否能直接通过工具解决问题或者计划。
            
如果需要拆分问题或者是计划，请输出以标题“计划：”开头的计划，然后是编号的步骤列表。 

否则不需要拆分，请直接输出<End>，不用返回任何内容。

工具：
{tools}



""",
        ),
        ("user", "{input}"),
        #MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

splitTaskPrompt = splitTaskPrompt.partial(
        tools=render_text_description(list(tools)),
)
splitTaskPrompt

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={'tools': 'word_length(word: str) -> int - 计算传入的单词的长度\nget_weather(location: str) -> dict - 根据城市获取天气数据\nsearch(keywords: str) -> str - 根据关键词使用DuckDuckGo搜索信息并返回'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['tools'], input_types={}, partial_variables={}, template='让我们首先了解用户问题或者是计划，并根据所提供的工具判断是否能直接通过工具解决问题或者计划。\n            \n如果需要拆分问题或者是计划，请输出以标题“计划：”开头的计划，然后是编号的步骤列表。 \n\n否则不需要拆分，请直接输出<End>，不用返回任何内容。\n\n工具：\n{tools}\n\n\n\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [22]:
splitTaskChain = splitTaskPrompt | llm | planParse
splitTaskChain.invoke({"input":"收集历任美国总统的个人信息和政治生涯信息（出生日期、就职日期、主要政策等）。"})

['使用search工具查询“历任美国总统”。', '从结果中提取每位总统的信息，包括他们的出生日期、就职日期以及他们担任总统期间实施的主要政策。']

In [23]:
splitTaskChain.invoke({"input":"搜索每个总统的名字，并查看他们的维基百科页面。"})

['使用search()函数搜索“美国总统名单”。',
 '从结果中提取所有美国总统的姓名列表。',
 '对于每个名字，使用search()函数搜索该人的维基百科页面。']